In [1]:
import sys  
!{sys.executable} -m pip install matplotlib python-dateutil

You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.2.0/libexec/bin/python3.8 -m pip install --upgrade pip' command.


In [2]:
import calendar
import datetime as dt
import dateutil.parser as dp
import pandas as pd
import matplotlib.pyplot as plt
from functools import partial

In [3]:
df = pd.read_json(r'data.json', convert_dates=True)
df["duration"] = (df["end_time"] - df["start_time"]).dt.days
df["count"] = 1
df.head()

q_75 = partial(pd.Series.quantile, q=0.75)
q_75.__name__ = "75%"

In [4]:
cancelled = df[(df["idList"] == "Cancelled")]
done = df[(df["idList"] == "Done")]

# Raw data...

In [5]:
df

,name,shortUrl,idList,start_time,end_time,duration,count
0,MCAFEE_EPO,https://trello.com/c/k60VLjOa,Done,2020-08-24 13:22:00+00:00,2020-09-25 17:33:22+00:00,32,1
1,BRO_JSON,https://trello.com/c/Eui55N6L,Done,2020-09-03 19:42:51+00:00,2020-09-24 14:31:15+00:00,20,1
2,DARKTRACE,https://trello.com/c/8wx0tVYa,Done,2020-09-01 19:11:55+00:00,2020-09-22 13:58:57+00:00,20,1
3,ABSOLUTE,https://trello.com/c/FVqlbJVA,Done,2020-08-28 15:17:54+00:00,2020-09-04 13:46:00+00:00,6,1
4,CLOUD_PASSAGE,https://trello.com/c/zvLnKRMt,Done,2020-08-19 16:58:00+00:00,2020-09-03 11:44:36+00:00,14,1
5,SOURCEFIRE_IDS,https://trello.com/c/zGAtO5Io,Done,2020-07-30 17:34:08+00:00,2020-09-02 12:58:46+00:00,33,1
6,Fix Attached Files and security_result to GMAI...,https://trello.com/c/aeMaPZDe,Done,2020-08-13 19:33:45+00:00,2020-08-19 16:31:27+00:00,5,1
7,AIRWATCH (log issues fixed),https://trello.com/c/XpFvP9Xm,Done,2020-07-28 15:33:23+00:00,2020-08-17 21:34:14+00:00,20,1
8,IPSWITCH_SFTP,https://trello.com/c/H76wnXs7,Done,2020-07-17 15:12:18+00:00,2020-08-07 19:00:13+00:00,21,1
9,Reduce significant alerts generated by GMAIL_LOGS,https://trello.com/c/OsPbu4Ht,Done,2020-07-30 16:38:13+00:00,2020-07-31 19:38:17+00:00,1,1


# Done estimate

In [6]:
done["duration"].describe()
done["duration"].agg([q_75])

75%    21.0
Name: duration, dtype: float64

# Trying another one: Done estimate (with "extremes" removed)

In [7]:
done_extremes_removed = done[(done["duration"] > 5) & (done["duration"] < 35)]
#done_extremes_removed.sort_values(by="duration")

In [8]:
done_extremes_removed["duration"].describe()
done_extremes_removed["duration"].agg([q_75])

75%    21.75
Name: duration, dtype: float64

# Trying another one: Now separating small from non-small

In [9]:
small_extremes_removed = done_extremes_removed[(done_extremes_removed["duration"] <= 10)]
not_small_extremes_removed = done_extremes_removed[(done_extremes_removed["duration"] > 10)]

In [10]:
small_extremes_removed["duration"].describe()
small_extremes_removed["duration"].agg([q_75])

75%    8.5
Name: duration, dtype: float64

In [11]:
not_small_extremes_removed["duration"].describe()
not_small_extremes_removed["duration"].agg([q_75])

75%    25.25
Name: duration, dtype: float64

In [12]:
small = done[(done["duration"] <= 6)]
not_small = done[(done["duration"] >= 6)]

# Another try: Don't remove extremes, but separate in small and non-small group

In [13]:
small["duration"].describe()
small["duration"].agg([q_75])

75%    4.75
Name: duration, dtype: float64

In [14]:
not_small["duration"].describe()
not_small["duration"].agg([q_75])

75%    23.5
Name: duration, dtype: float64

# Ok, let's go with 4.75 for small ones, and 23.5 for non-small ones.

## 5 weekdays for small ones

## 17 weekdays (23 days - 6 weekends) for non-small ones